# Sample for KFServing SDK v1beta1

This is a sample for KFServing SDK v1beta1. 

The notebook shows how to use KFServing SDK to create, get and delete InferenceService.

***!!!***

***Kfserving depends on azure-storage-blob 2.1.0.***

***This will break some of the ydata packages.***

***Please check the kfserving from pipelines laboratory.***

In [ ]:
!pip install azure-storage-blob==2.1.0

In [ ]:
from kubernetes import client 
from kfserving import KFServingClient
from kfserving import constants
from kfserving import utils
from kfserving import V1beta1InferenceService
from kfserving import V1beta1InferenceServiceSpec
from kfserving import V1beta1PredictorSpec
from kfserving import V1beta1TFServingSpec

Define namespace where InferenceService needs to be deployed to. If not specified, below function defines namespace to the current one where SDK is running in the cluster, otherwise it will deploy to default namespace.

In [ ]:
namespace = utils.get_default_target_namespace()
inferenceServiceName = "flower-sample1"

## Define InferenceService

Firstly define default endpoint spec, and then define the inferenceservice basic on the endpoint spec.

In [ ]:
api_version = constants.KFSERVING_GROUP + "/v1beta1"

annotations = {}
annotations["sidecar.istio.io/inject"] = "false"
isvc = V1beta1InferenceService(api_version=api_version,
                               kind=constants.KFSERVING_KIND,
                               metadata=client.V1ObjectMeta(
                                   name=inferenceServiceName, namespace=namespace, annotations=annotations),
                               spec=V1beta1InferenceServiceSpec(
                               predictor=V1beta1PredictorSpec(
                               tensorflow=(V1beta1TFServingSpec(
                                   storage_uri='gs://kfserving-samples/models/tensorflow/flowers'))))
)

## Create InferenceService

Call KFServingClient to create InferenceService.

In [ ]:
KFServing = KFServingClient()
KFServing.create(isvc)

## Check the InferenceService

In [ ]:
KFServing.wait_isvc_ready(inferenceServiceName, namespace=namespace)
KFServing.get(inferenceServiceName, namespace=namespace, watch=False, timeout_seconds=120)

## Patch the InferenceService and define Canary Traffic Percent

In [ ]:

isvc = V1beta1InferenceService(api_version=api_version,
                               kind=constants.KFSERVING_KIND,
                               metadata=client.V1ObjectMeta(
                                   name=inferenceServiceName, namespace=namespace, annotations=annotations),
                               spec=V1beta1InferenceServiceSpec(
                               predictor=V1beta1PredictorSpec(
                                   canary_traffic_percent=20,
                                   tensorflow=(V1beta1TFServingSpec(
                                       storage_uri='gs://kfserving-samples/models/tensorflow/flowers-2'))))
)

KFServing.patch(inferenceServiceName, isvc, namespace=namespace)

### Check the InferenceService after Patching

In [ ]:
KFServing.wait_isvc_ready(inferenceServiceName, namespace=namespace)

In [ ]:
KFServing.get(inferenceServiceName, namespace=namespace, watch=False)

## Delete the InferenceService

In [ ]:
KFServing.delete(inferenceServiceName, namespace=namespace)